In [1]:
from model import TwinLite as net
model = net.TwinLiteNet()

In [2]:
model

TwinLiteNet(
  (encoder): ESPNet_Encoder(
    (level1): CBR(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=16)
    )
    (sample1): InputProjectionA(
      (pool): ModuleList(
        (0): AvgPool2d(kernel_size=3, stride=2, padding=1)
      )
    )
    (sample2): InputProjectionA(
      (pool): ModuleList(
        (0-1): 2 x AvgPool2d(kernel_size=3, stride=2, padding=1)
      )
    )
    (b1): CBR(
      (conv): Conv2d(19, 19, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(19, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=19)
    )
    (level2_0): DownSamplerB(
      (c1): C(
        (conv): Conv2d(19, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      )
      (d1): CDilated(
        (conv): Conv2d(12, 16,

In [3]:
import torch
checkpoint = torch.load("./pretrained/best.pth")
state_dict = {}
for key in checkpoint:
    state_dict[key.replace("module.","")] = checkpoint[key]
model.load_state_dict(state_dict)

<All keys matched successfully>

In [24]:
import torch.nn as nn
def replace_activations(model):
    """
    Recursively replaces PReLU after nn.Conv2d layers but retains them after nn.ConvTranspose2d layers.
    Also replaces Softmax with ReLU.
    """
    for name, child in model.named_children():
        if isinstance(child, nn.Conv2d):
            # Check the next child to see if it's PReLU
            next_child = next(next(model.children()))
            # next_child = next(model.children()) # Batch Norm
            
            if isinstance(next_child, nn.PReLU):
                # Replace PReLU with ReLU
                setattr(model, name + "_relu", nn.ReLU(inplace=True))
                delattr(model, name)  # Remove the original PReLU
        elif isinstance(child, nn.Softmax):
            # Replace Softmax with ReLU
            setattr(model, name, nn.ReLU(inplace=True))
        elif isinstance(child, nn.Sequential):
            # Recursive call for nested modules like nn.Sequential
            replace_activations(child)
    
    return model

def replace_prelu_with_relu(model):
    #model = model.copy()
    for name, child in model.named_children():
        # Check if the current module is PReLU
        if isinstance(child, nn.PReLU):
            setattr(model, name, nn.ReLU(inplace=True))
        # Call the function recursively for child modules
        replace_prelu_with_relu(child)
replace_activations(model)
model

TwinLiteNet(
  (encoder): ESPNet_Encoder(
    (level1): CBR(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=16)
    )
    (sample1): InputProjectionA(
      (pool): ModuleList(
        (0): AvgPool2d(kernel_size=3, stride=2, padding=1)
      )
    )
    (sample2): InputProjectionA(
      (pool): ModuleList(
        (0-1): 2 x AvgPool2d(kernel_size=3, stride=2, padding=1)
      )
    )
    (b1): CBR(
      (conv): Conv2d(19, 19, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(19, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=19)
    )
    (level2_0): DownSamplerB(
      (c1): C(
        (conv): Conv2d(19, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      )
      (d1): CDilated(
        (conv): Conv2d(12, 16,

In [5]:
model

TwinLiteNet(
  (encoder): ESPNet_Encoder(
    (level1): CBR(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=16)
    )
    (sample1): InputProjectionA(
      (pool): ModuleList(
        (0): AvgPool2d(kernel_size=3, stride=2, padding=1)
      )
    )
    (sample2): InputProjectionA(
      (pool): ModuleList(
        (0-1): 2 x AvgPool2d(kernel_size=3, stride=2, padding=1)
      )
    )
    (b1): CBR(
      (conv): Conv2d(19, 19, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(19, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): PReLU(num_parameters=19)
    )
    (level2_0): DownSamplerB(
      (c1): C(
        (conv): Conv2d(19, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      )
      (d1): CDilated(
        (conv): Conv2d(12, 16,

In [6]:
#tensor = torch.randint(low=0, high=256, size=(1, 3, 320, 160), dtype=torch.uint8)
tensor = torch.rand(size=(1, 3, 320, 320))

# ImageNet mean and std values for normalization
means = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
stds = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)

# Normalize the tensor
normalized_tensor = (tensor - means) / stds

In [7]:
!pip install nobuco

Looking in indexes: https://pypi.org/simple, http://pypi.private.ebuild.esmartcontrol.net/simple


In [8]:
model(normalized_tensor)

(tensor([[[[ 2.7293,  2.7028,  2.7432,  ...,  2.4589,  2.5283,  2.5167],
           [ 2.7991,  2.7725,  2.8137,  ...,  2.4844,  2.5597,  2.5479],
           [ 2.7450,  2.7162,  2.7598,  ...,  2.4030,  2.4747,  2.4649],
           ...,
           [-2.3489, -2.2584, -2.3339,  ..., -3.4228, -3.3929, -3.2536],
           [-2.2593, -2.1688, -2.2457,  ..., -3.1604, -3.1276, -2.9919],
           [-2.3883, -2.2981, -2.3754,  ..., -3.2365, -3.1994, -3.0643]],
 
          [[-6.3829, -6.3185, -6.4078,  ..., -5.8796, -6.0166, -5.9829],
           [-6.5090, -6.4427, -6.5352,  ..., -5.9316, -6.0799, -6.0444],
           [-6.4102, -6.3410, -6.4369,  ..., -5.7776, -5.9183, -5.8884],
           ...,
           [ 0.8604,  0.7911,  0.8510,  ...,  1.4224,  1.4339,  1.3298],
           [ 0.8064,  0.7378,  0.7978,  ...,  1.2719,  1.2794,  1.1791],
           [ 0.8875,  0.8184,  0.8793,  ...,  1.3221,  1.3283,  1.2273]]]],
        grad_fn=<PreluKernelBackward0>),
 tensor([[[[ 2.1133,  2.0914,  2.1592,  ..., 

In [9]:
normalized_tensor.shape

torch.Size([1, 3, 320, 320])

In [10]:
import nobuco
from nobuco import ChannelOrder, ChannelOrderingStrategy
from nobuco.layers.weight import WeightLayer

2023-11-02 14:56:09.026460: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-02 14:56:09.051760: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 14:56:09.051781: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 14:56:09.051799: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-02 14:56:09.057138: I tensorflow/core/platform/cpu_feature_g

In [11]:
dummy_image = torch.rand(size=(1, 3, 360, 640))
pytorch_module = model

keras_model = nobuco.pytorch_to_keras(
    pytorch_module.eval(),
    args=[dummy_image], kwargs=None,
    inputs_channel_order=ChannelOrder.TENSORFLOW,
    outputs_channel_order=ChannelOrder.TENSORFLOW
)
keras_model.save('twinlitenet_tf')

2023-11-02 14:56:11.494397: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-02 14:56:11.525635: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-02 14:56:11.525760: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Legend:
    Green — conversion successful
    Yellow — conversion imprecise
    Red — conversion failed
    Red — no converter found
    Bold — conversion applied directly
    * — subgraph reused
    Tensor — this output is not dependent on any of subgraph's input tensors
    Tensor — this input is a parameter / constant
    Tensor — this tensor is useless

 (!) Max diff 0.006401  
 I  File "/home/raphael/esmart/TwinLiteNet/venv/lib/python3.10/site-packages/nobuco/trace/trace.py", line 244 
 D  File "/home/raphael/esmart/TwinLiteNet/model/TwinLite.py", line 433  
TwinLiteNet[model.TwinLite](float32_0<1,3,360,640>) -> (float32_299<1,2,360,640>, float32_326<1,2,360,640>)
 │  ESPNet_Encoder[model.TwinLite](float32_0<1,3,360,640>) -> float32_272<1,32,45,80>
 │   │   (!) Max diff 0.000318  
 │   │   I  File "/home/raphael/esmart/TwinLiteNet/model/TwinLite.py", line 401 
 │   │   D  File "/home/raphael/esmart/TwinLiteNet/model/TwinLite.py", line 103  
 │   │  CBR[model.TwinLite](float32_0<1,

INFO:tensorflow:Assets written to: twinlitenet_tf/assets


In [12]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, http://pypi.private.ebuild.esmartcontrol.net/simple


In [13]:
torch.onnx.export(model,               # model being run
                  normalized_tensor, "twinlitenet.onnx", input_names=["input"], output_names=["drivable_area","lanes"])

In [14]:
#!pip install tensorflow
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("twinlitenet_tf")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()


2023-11-02 14:56:26.407181: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-02 14:56:26.407202: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-02 14:56:26.407713: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: twinlitenet_tf
2023-11-02 14:56:26.424267: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-02 14:56:26.424289: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: twinlitenet_tf
2023-11-02 14:56:26.454476: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-11-02 14:56:26.464330: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-02 14:56:26.651083: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: twinlitenet_tf
2023-11-02 14

In [15]:
with open('quantized_model.tflite', 'wb') as f:
    f.write(tflite_quant_model)


In [16]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("twinlitenet_tf")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()
with open('quantized_model_float16.tflite', 'wb') as f:
    f.write(tflite_quant_model)

2023-11-02 14:56:29.427656: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-02 14:56:29.427680: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-02 14:56:29.427855: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: twinlitenet_tf
2023-11-02 14:56:29.440569: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-02 14:56:29.440617: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: twinlitenet_tf
2023-11-02 14:56:29.479049: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-02 14:56:29.676790: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: twinlitenet_tf
2023-11-02 14:56:29.770705: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 342850 micros

In [17]:
import glob, cv2
images = glob.glob("../esmart-ai-datasets/data/esmart_wip/*.jpg")
images

['../esmart-ai-datasets/data/esmart_wip/20210630_192517_9870.jpg',
 '../esmart-ai-datasets/data/esmart_wip/Log-20220218-091055 Data Log_25622.jpg',
 '../esmart-ai-datasets/data/esmart_wip/Log-20220121-134942 Data Log_23249.jpg',
 '../esmart-ai-datasets/data/esmart_wip/20210711_110911_5814.jpg',
 '../esmart-ai-datasets/data/esmart_wip/Log-20220218-091055 Data Log_4644.jpg',
 '../esmart-ai-datasets/data/esmart_wip/20211130_080737_39139.0.jpg',
 '../esmart-ai-datasets/data/esmart_wip/Log-20220121-143049 Data Log_7021.jpg',
 '../esmart-ai-datasets/data/esmart_wip/Log-20220414-064154 Data Log_91111.jpg',
 '../esmart-ai-datasets/data/esmart_wip/20210711_174204_26544.jpg',
 '../esmart-ai-datasets/data/esmart_wip/Log-20220123-124034 Data Log_18487.jpg',
 '../esmart-ai-datasets/data/esmart_wip/Log-20221025-173346 Data Log_135845.jpg',
 '../esmart-ai-datasets/data/esmart_wip/Log-20220222-083904 Data Log_40479.jpg',
 '../esmart-ai-datasets/data/esmart_wip/Log-20221027-181018 Data Log_33117.jpg',


In [18]:
def representative_dataset_gen():
    # Representative dataset for use with converter.representative_dataset
    images = glob.glob("../esmart-ai-datasets/data/esmart_wip/*.jpg")
    for image in images[0:10]:
        # Get sample input data as a numpy array in a method of your choosing.
        img = cv2.imread(image)
        img = cv2.resize(img, (640, 360))
        img = np.expand_dims(img, axis=0).astype(np.float32)
        img /= 255.0
        yield [img]
            

In [19]:
import tensorflow as tf
import numpy as np
converter = tf.lite.TFLiteConverter.from_saved_model("twinlitenet_tf")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
#converter.inference_input_type = tf.int8  # or tf.uint8
#converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()
with open('quantized_model_int8_static.tflite', 'wb') as f:
    f.write(tflite_quant_model)

2023-11-02 14:56:32.548197: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-02 14:56:32.548219: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-02 14:56:32.548354: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: twinlitenet_tf
2023-11-02 14:56:32.562380: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-02 14:56:32.562401: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: twinlitenet_tf
2023-11-02 14:56:32.597595: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-02 14:56:32.772258: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: twinlitenet_tf
2023-11-02 14:56:32.851612: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 303257 micros

In [20]:
tflite_quant_model

b' \x00\x00\x00TFL3\x00\x00\x00\x00\x14\x00 \x00\x1c\x00\x18\x00\x14\x00\x10\x00\x0c\x00\x00\x00\x08\x00\x04\x00\x14\x00\x00\x00\x1c\x00\x00\x00\xac\x00\x00\x00\x04\x01\x00\x004\xcf\x06\x00D\xcf\x06\x00\x9c%\x08\x00\x03\x00\x00\x00\x01\x00\x00\x00\x04\x00\x00\x00\xf2\x04\xf9\xff\x0c\x00\x00\x00\x1c\x00\x00\x00d\x00\x00\x00\x0f\x00\x00\x00serving_default\x00\x02\x00\x00\x00(\x00\x00\x00\x04\x00\x00\x00t\xff\xff\xffs\x01\x00\x00\x04\x00\x00\x00\r\x00\x00\x00tf.identity_4\x00\x00\x00\x94\xff\xff\xffl\x01\x00\x00\x04\x00\x00\x00\r\x00\x00\x00tf.identity_3\x00\x00\x00\x01\x00\x00\x00\x04\x00\x00\x00\x8e\x05\xf9\xff\x04\x00\x00\x00\x06\x00\x00\x00args_0\x00\x00\x02\x00\x00\x004\x00\x00\x00\x04\x00\x00\x00\xdc\xff\xff\xffv\x01\x00\x00\x04\x00\x00\x00\x13\x00\x00\x00CONVERSION_METADATA\x00\x08\x00\x0c\x00\x08\x00\x04\x00\x08\x00\x00\x00u\x01\x00\x00\x04\x00\x00\x00\x13\x00\x00\x00min_runtime_version\x00w\x01\x00\x00,\xce\x06\x00$\xce\x06\x00\xf4\xcd\x06\x00\xc4\xcd\x06\x00\x94\xcd\x06\x00d\xcd

In [21]:
with open('quantized_model_int8_static.tflite', 'wb') as f:
    f.write(tflite_quant_model)

In [22]:
import tensorflow as tf

# Load the model
converter = tf.lite.TFLiteConverter.from_saved_model("twinlitenet_tf")

# By default, the converter will use float32 precision.
tflite_model = converter.convert()

# Save the TFLite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

2023-11-02 14:56:43.359885: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-02 14:56:43.359907: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-02 14:56:43.360033: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: twinlitenet_tf
2023-11-02 14:56:43.375246: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-02 14:56:43.375265: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: twinlitenet_tf
2023-11-02 14:56:43.411871: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-02 14:56:43.591628: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: twinlitenet_tf
2023-11-02 14:56:43.670768: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 310735 micros